In [1]:
from urllib.request import *
from urllib import request
import urllib.parse
from bs4 import BeautifulSoup
import bs4
import re
import numpy as np
import requests
from tika import parser as tkparser
import lxml
from selenium import webdriver
import time
import pandas as pd
# import Exception

In [17]:
class crawler():
    def __init__(self):
        self.papers = pd.DataFrame(columns=['title', 'keywords_lang', 'keywords', 'abstract_lang','abstract'])
        self.df_idx = 0 
        self.max_page_view = 20
        self.basic_url = 'http://kmbase.medric.or.kr'
        self.browser = webdriver.Chrome('C:/Users/NB-0242/jupyter/PHR/chromedriver.exe')
        self.browser.get('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=SEARCH&s=NORMAL&kw=%ub300%uc0ac%uc99d%ud6c4%uad70')
        
#         search_html = self.browser.page_source
#         search_bs = BeautifulSoup(search_html, 'html.parser')
#         self.n_papers = int(search_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_total_count'}).find_all('font')[1].get_text())
    
    
    def page_crawling(self):
        for view in range(3):
            self.page_filler()
            if view ==0:
                for i in range(2, self.max_page_view+2):
                    print('view {} , {}th iter start'.format(view, i))
                    self.browser.find_element_by_xpath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[{}]/a'.format(i)).click()
                    time.sleep(3)
                    self.page_filler()
                    
            elif view==1:
                for i in range(3, self.max_page_view+3):
                    print('view {} , {}th iter start'.format(view, i))
                    self.browser.find_element_by_xpath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[{}]/a'.format(i)).click()

                    time.sleep(3)
                    self.page_filler()
            elif view==2:
                for i in range(17, self.max_page_view+2):
                    print('view {} , {}th iter start'.format(view, i))
                    self.browser.find_element_by_xpath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[{}]/a'.format(i)).click()                   
                    time.sleep(3)
                    self.page_filler()                
#             self.browser.find_element_by_expath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[21]/a')
            print('{}page filler finished'.format(view))
        
    
    def get_query_links(self, bs_object):
        query_links = bs_object.find('div',{'id' : 'mainWrapper'}).find('table', {'id' : 'ctl11_ctl01_ctl00_KMbase_table'}).find_all('li')
        return query_links
    
    def get_filtered_links(self, query_links):
        query_links_filtered =[]
        for idx, link in enumerate(query_links) :
            refined = str(link).strip('<li>').strip('</li>').strip()

            if re.compile('<a').match(refined):
                splited = refined.split('>')[0].strip('<a href=').strip('"')
                if re.compile('http').match(splited):
                    continue
                else:
                    query_links_filtered.append(splited)   
        return np.unique(query_links_filtered)
    
    def get_query_bs(self, filtered_link):
        query_addr = basic_url + filtered_link.replace('amp;', '')
        query_html = urlopen(query_addr)
        query_bs = BeautifulSoup(query_html, 'html.parser')
        journal_info = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')
        return query_bs, journal_info

    def make_keywords_str(self, keywords):
        keywords_str = ''
        for i in range(len(keywords)):
            keywords_str += keywords[i] + ','
        return keywords_str
    
    
    
    def page_filler(self):
        search_html = self.browser.page_source
        search_bs = BeautifulSoup(search_html, 'html.parser')
        search_links = self.get_query_links(search_bs)
        search_links_filtered = self.get_filtered_links(search_links)

        for i in range( len( search_links_filtered)):
            self.article_addr = self.basic_url + str(search_links_filtered[i]).replace('amp;', '')
            try : 
                article_bs = BeautifulSoup(urlopen(self.article_addr), 'html.parser')
                
            except:
                continue
            
            try :
                title = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_title_kor'}).get_text()
            except : 
                continue
                
#             print(title)
            abstract_lang = 'en'
            keywords_lang = 'en'
            try : 
                if bool(article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_kor'}).get_text()):
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_kor'}).get_text()
                    abstract_lang = 'ko'
                else:
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_eng'}).get_text()

            except :
                try :
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_eng'}).get_text()
                except : 
                    abstract = [""]
                


            try:
                if bool(article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_kor' }).get_text()):
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_kor' }).get_text().split(';')
                    keywords_lang ='ko'
                else:
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_eng' }).get_text().split(';')

            except : 
                try : 
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_eng' }).get_text().split(';')
                    
                except :
                    keywords = [""]
                    
            keywords_str = self.make_keywords_str(keywords)

            self.papers.loc[self.df_idx] = title, keywords_lang, keywords_str , abstract_lang, abstract
            self.df_idx +=1
             

In [18]:
crawler = crawler()

In [19]:
crawler.page_crawling()

view 0 , 2th iter start
view 0 , 3th iter start
view 0 , 4th iter start
view 0 , 5th iter start
view 0 , 6th iter start
view 0 , 7th iter start
view 0 , 8th iter start
view 0 , 9th iter start
view 0 , 10th iter start
view 0 , 11th iter start
view 0 , 12th iter start
view 0 , 13th iter start
view 0 , 14th iter start
view 0 , 15th iter start
view 0 , 16th iter start
view 0 , 17th iter start
view 0 , 18th iter start
view 0 , 19th iter start
view 0 , 20th iter start
view 0 , 21th iter start
0page filler finished
view 1 , 3th iter start
view 1 , 4th iter start
view 1 , 5th iter start
view 1 , 6th iter start
view 1 , 7th iter start
view 1 , 8th iter start
view 1 , 9th iter start
view 1 , 10th iter start
view 1 , 11th iter start
view 1 , 12th iter start
view 1 , 13th iter start
view 1 , 14th iter start
view 1 , 15th iter start
view 1 , 16th iter start
view 1 , 17th iter start
view 1 , 18th iter start
view 1 , 19th iter start
view 1 , 20th iter start
view 1 , 21th iter start
view 1 , 22th iter

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[22]/a"}
  (Session info: chrome=80.0.3987.163)


In [20]:
crawler.papers

,title,keywords_lang,keywords,abstract_lang,abstract
0,한국 성인의 식생활평가지수에 기반한 전체 식사의 질과 대사증후군 구성요서 및 대사증...,en,"diet quality, Korean Healthy Eating Index, met...",en,Objectives: This study examined the associatio...
1,폐경 경험 및 대사증후군 구성요소와 치주염의 관련성,en,"Menopause status, Metabolic syndrome, Oral hea...",en,Objectives: The purpose of this study was to i...
2,대사증후군 요소와 치주염의 관련성: 35-79세 한국인,en,"Epidemiology, Metabolic syndrome, Periodontitis,",en,Objectives: The purpose of this study was to i...
3,계획된 행위이론에 근거한 대사증후군을 진단받은 대상자의 건강행위 설명모형,ko,"대사증후군, 의도, 가족의 지지, 건강행위,",en,Purpose: This study aimed to identify the fact...
4,대사증후군을 가진 조현병 환자에서 활동기록기로 평가한 수면의 질과 신체 활동량,ko,"조현병·대사증후군, 활동기록기, 수면의 질, 신체 활동,",en,Objectives：The authors examined the associatio...
...,...,...,...,...,...
947,비비만인에서 발견되는 비알콜성 지방간의 대사적 의미,ko,"비알콜성 지방간, 체질량지수, 대사증후군, 중심성 비만, Non-alcoholic ...",ko,"목적: 비알콜성 지방간은 비만, 당뇨병 그리고 지질대사 이상 등과 관련성이 깊다고 ..."
948,"한국 성인에서 흡연, 음주, 운동, 교육정도 및 가족력이 대사증후군에 미치는 영향",en,"Smoking, Alcohol, Exercise, Family history, Ed...",ko,배경: 대사증후군은 심혈관 질환의 이환율과 이로 인한 사망률을 증가시키는 것으로 알...
949,체중 감소의 득과 실：올바른 다이어트하기,ko,"체중감소,효과,올바른 다이어트,",ko,비만이 각종 성인병 및 소위 말하는 대사증후군의 위험성을 올려 주어 건강상 많은 문...
950,한국인에서의 대사증후군의 유병률 -일개 대학병원 종합검진센터 수진자를 대상으로-,ko,"대사증후군, 유병률, 체질량지수, 중성지방, 혈압, 혈당, metabolic syn...",ko,"연구배경: 대사증후군이란 고중성지방혈증, 고혈압, 당대사이상 및 비만과 같은 관상동..."


In [24]:
crawler.papers.drop_duplicates(inplace=True)

In [27]:
crawler.papers.to_csv('papers_abs/대사증후군_초록.csv', index=False)

In [90]:
papers = crawler.papers

In [45]:
# crawler.papers.query('keywords_lang=="en" | abstract_lang=="en"')

,title,keywords_lang,keywords,abstract_lang,abstract
0,한국 성인의 식생활평가지수에 기반한 전체 식사의 질과 대사증후군 구성요서 및 대사증...,en,"diet quality, Korean Healthy Eating Index, met...",en,Objectives: This study examined the associatio...
1,폐경 경험 및 대사증후군 구성요소와 치주염의 관련성,en,"Menopause status, Metabolic syndrome, Oral hea...",en,Objectives: The purpose of this study was to i...
2,대사증후군 요소와 치주염의 관련성: 35-79세 한국인,en,"Epidemiology, Metabolic syndrome, Periodontitis,",en,Objectives: The purpose of this study was to i...
3,계획된 행위이론에 근거한 대사증후군을 진단받은 대상자의 건강행위 설명모형,ko,"대사증후군, 의도, 가족의 지지, 건강행위,",en,Purpose: This study aimed to identify the fact...
4,대사증후군을 가진 조현병 환자에서 활동기록기로 평가한 수면의 질과 신체 활동량,ko,"조현병·대사증후군, 활동기록기, 수면의 질, 신체 활동,",en,Objectives：The authors examined the associatio...
...,...,...,...,...,...
944,대사증후군의 위험예측인자로서 인슐린저항성,en,"Insulin resistance, Metabolic syndrome, Cardio...",ko,목적: 본 연구는 한국인 성인에서 인슐린저항성에 따른 대사증후군과 그 구성요소들의 ...
945,한국인 성인에서 내장지방량과 죽상경화증 위험인자 및 좌심실 비대와의 상관관계,en,"Obesity, Visceral adipose tissue, LVH,,",ko,"배경: 비만증은 심혈관 질환의 위험요소일 뿐 아니라 고혈압, 이상지질혈증, 인슐린 ..."
946,"도시지역 검진수진자 40,698명에서 NCEP-APTⅢ 기준에 의한 대사증후군 유병률",en,"Prevalence, Metabolic sydrome, NCEP ATP-Ⅲ,,",ko,목적: 대사증후군은 그 진단기준의 다양함으로 인해 이 질환 군의 통계학적인 정보나 ...
948,"한국 성인에서 흡연, 음주, 운동, 교육정도 및 가족력이 대사증후군에 미치는 영향",en,"Smoking, Alcohol, Exercise, Family history, Ed...",ko,배경: 대사증후군은 심혈관 질환의 이환율과 이로 인한 사망률을 증가시키는 것으로 알...


## 번역 API 

In [38]:
# ! pip install google-cloud-translate==2.0.0
! pip install googletrans

  Created wheel for googletrans: filename=googletrans-2.4.0-py3-none-any.whl size=15779 sha256=016bc0dd91d6f835d29f4f645bb3774196efa77b7fa618d04669fda0a09f3038
  Stored in directory: c:\users\nb-0242\appdata\local\pip\cache\wheels\29\2c\06\957b5a801b311507090697dde39473f3d70154f1ca7142efa8
Successfully built googletrans


In [57]:
from tqdm import tqdm_notebook

In [39]:
from googletrans import Translator
translator = Translator()

In [103]:
papers.reset_index(drop=True, inplace=True)

In [121]:
keywords_dic = {}
abstract_dic = {}
error_dic = {}
for idx ,row in tqdm_notebook(papers.iterrows()):
    if (row['keywords_lang']=='en'):
        try:
            keywords_trans = translator.translate(row['keywords'],src='en', dest='ko').text
    #         papers.loc[idx]['keywords'] = keywords_trans
            keywords_dic[idx] = keywords_trans
        except :
            error_dic[idx] = 'keywords'
            
    if (row['abstract_lang']=='en'):
        try:
            abstract_trans = translator.translate(row['abstract'], src='en', dest='ko').text
            papers.loc[idx]['abstract'] = abstract_trans
            abstract_dic[idx] = abstract_trans
        except :
            error_dic[idx] = 'abstract'

C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [151]:
# 특별한 이유없이 트랜잭션 병목현상으로 인해 JSON 에러가남 -> SLEEP 조금 줌 ㅎ
for idx, columns in tqdm_notebook(zip(error_dic.keys(), error_dic.values())):
    trans_res = translator.translate(papers.loc[idx][columns],src='en', dest='ko').text
    papers.loc[idx][columns] = trans_res
    time.sleep(0.5)

C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [156]:
papers.loc[50:100]['abstract'][55]

'WHtR (높이 비율 몸통) 및 ABSI (a 체형 지수) 복부 비만을 반영하는 지표이다. 이 연구는 인슐린 저항성과 ABSI 및 WHtR의 대사 증후군의 예측 능력을 조사 하였다. 본 연구에서는 3 천 4 백 95 명 20 년에 종합 병원에서 신체 검사를받은 사람, 나이가 세 경기를 -do 년 1 월 9 월 가입일 2,017 2,017 단면 조사에 평가되었다. 대사 증후군은 AHA / NHLBI의 기준에 따라 정의 하였다. 인슐린 저항성은 HOMA-IR 값이 3.0 이상인 경우 인슐린 저항성이라고 판정 하였다. 남성과 여성 모두가 WHtR 및 ABSI보다 대사 위험 요인 사이에 강한 상관 관계를 보였다. WHtR 및 ABSI의 AUC 값 (p <0.001)는 각각 0.849 및 0.676이었다. 인슐린 저항성을 예측하고 WHtR ABSI의 AUC 값 (p <0.001)는 각각 0.818 및 0.641이었다. 결론적 ABSI 인슐린 저항과 WHtR 대사 증후군, 인슐린 저항성에 대한 좋은 예측 능력을 갖는 반면, 대사 증후군의 낮은 예측력을 갖는다.'

In [158]:
papers.drop(columns=['keywords_lang', 'abstract_lang'], inplace=True)

In [160]:
papers.to_csv('papers_abs/대사증후군_초록_번역.csv', index=False)